In [1]:
import glob
import os
import sys

In [2]:
try:
    sys.path.append(glob.glob('/opt/carla-simulator/PythonAPI/carla/dist/carla-*3.7-%s.egg' % (
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

In [3]:
import carla

In [4]:
import random
import time
import numpy as np
import cv2

In [5]:
IM_WIDTH = 640
IM_HEIGHT = 480
actor_list = []

In [6]:
def process_img(image):
    i = np.array(image.raw_data)
    i2 = i.reshape((IM_HEIGHT, IM_WIDTH, 4))
    i3 = i2[:, :, :3]
#     cv2.imshow("image", i3)
#     cv2.waitKey(1)  
    return i3/255.0

In [21]:
client = carla.Client("localhost", 2000)
client.set_timeout(2.0)
world = client.get_world()
blueprint_library = world.get_blueprint_library()

bp = blueprint_library.filter("model3")[0]
print(bp)

spawn_point = random.choice(world.get_map().get_spawn_points())

vehicle = world.spawn_actor(bp, spawn_point)
#     vehicle.set_autopilot(True)
vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0.0))
actor_list.append(vehicle)

cam_bp = blueprint_library.find("sensor.camera.rgb")
cam_bp.set_attribute("image_size_x", f"{IM_WIDTH}")
cam_bp.set_attribute("image_size_y", f"{IM_HEIGHT}")
cam_bp.set_attribute("fov", "110")

spawn_point = carla.Transform(carla.Location(x=2.5, z=0.7))

sensor = world.spawn_actor(cam_bp, spawn_point, attach_to=vehicle)
actor_list.append(sensor)
sensor.listen(lambda data: process_img(data))

vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
time.sleep(4)

ActorBlueprint(id=vehicle.tesla.model3,tags=[model3, vehicle, tesla])


In [22]:
print(world.get_actors().filter("*vehicle*"))

[Actor(id=208, type=vehicle.tesla.model3)]


In [36]:
for actor in world.get_actors().filter("*vehicle*"):
    actor.destroy()

In [24]:
map = world.get_map()

In [25]:
num_CAVs = 5

In [26]:
spawn_points = []
if map.name == 'Town05':
    # These are three segments of straight road that I thought would be great spawn zones
    for x in np.linspace(-134, 100, 4):
        for y in (201, 205, 209):
            transform = map.get_waypoint(carla.Location(x=x, y=y)).transform
            transform.location.z = transform.location.z + 2.0
            spawn_points.append(transform)

    for y in np.linspace(95, -87, 3):
        for x in (211, 208, 204):
            transform = map.get_waypoint(carla.Location(x=x, y=y)).transform
            transform.location.z = transform.location.z + 2.0
            spawn_points.append(transform)

    for x in np.linspace(-134, 80, 4):
        for y in (-200, -204, -208):
            transform = map.get_waypoint(carla.Location(x=x, y=y)).transform
            transform.location.z = transform.location.z + 2.0
            spawn_points.append(transform)
else:
    spawn_points = map.get_spawn_points()
spawn_points = random.sample(spawn_points, num_CAVs)

In [33]:
for i in range(num_CAVs):
    transform = spawn_points[i]
    vehicle = world.spawn_actor(bp, transform)
    vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
    actor_list.append(vehicle)

    # add rgba camera for this vehicle
    transform = carla.Transform(carla.Location(x=2.5, z=0.7))
    sensor = world.spawn_actor(cam_bp, transform, attach_to=vehicle)
    actor_list.append(sensor)

In [28]:
print(world.get_actors().filter("*vehicle*"))

[Actor(id=210, type=vehicle.tesla.model3), Actor(id=212, type=vehicle.tesla.model3), Actor(id=214, type=vehicle.tesla.model3), Actor(id=216, type=vehicle.tesla.model3), Actor(id=218, type=vehicle.tesla.model3)]


In [31]:
print(world.get_actors().filter("*sensor*"))

[Actor(id=201, type=sensor.camera.rgb), Actor(id=203, type=sensor.camera.rgb), Actor(id=205, type=sensor.camera.rgb), Actor(id=207, type=sensor.camera.rgb), Actor(id=209, type=sensor.camera.rgb), Actor(id=211, type=sensor.camera.rgb), Actor(id=213, type=sensor.camera.rgb), Actor(id=215, type=sensor.camera.rgb), Actor(id=217, type=sensor.camera.rgb), Actor(id=219, type=sensor.camera.rgb)]
